In [17]:
import pymongo
from pymongo import MongoClient
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import string
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
import gensim
from gensim.models import Word2Vec
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\edouc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\edouc\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [13]:
#MongoDB connection
client = MongoClient("mongodb://localhost:27017")
db = client.wikidb
collection = db['wikipedia']

In [14]:
#Storing all text content in the database in a list
query = collection.find({},{'text':1, '_id':0})
liste = list(query)

In [48]:
#Clean up characters that do not have a place in the ascii table and give errors when printing and punctuation
i = ""
for item in liste:
    i += item['text'].encode("ascii", errors="ignore").decode()
    
i = i.lower()

for character in i:
    if character in string.punctuation:
        i = i.replace(character,"")

print(len(i))

1858924


In [66]:
#lemmatizing words into dictionary forms
lemmatizer = WordNetLemmatizer()
text_tokens = word_tokenize(i)
tokens_without_sw = [word for word in text_tokens if word not in STOPWORDS]

temp=""
for item in tokens_without_sw:
    temp += lemmatizer.lemmatize(item) + " "

#lemmatized words
tokens = word_tokenize(temp)

print(tokens)

['nan', 'agentbased', 'model', 'abm', 'computational', 'model', 'simulating', 'action', 'interaction', 'autonomous', 'agent', 'individual', 'collective', 'entity', 'organization', 'group', 'order', 'understand', 'behavior', 'system', 'governs', 'outcome', 'combine', 'element', 'game', 'theory', 'complex', 'system', 'emergence', 'computational', 'sociology', 'multiagent', 'system', 'evolutionary', 'programming', 'monte', 'carlo', 'method', 'used', 'understand', 'stochasticity', 'model', 'particularly', 'within', 'ecology', 'abm', 'called', 'individualbased', 'model', 'ibms1', 'review', 'recent', 'literature', 'individualbased', 'model', 'agentbased', 'model', 'multiagent', 'system', 'show', 'abm', 'used', 'many', 'scientific', 'domain', 'including', 'biology', 'ecology', 'social', 'science2', 'agentbased', 'modeling', 'related', 'distinct', 'concept', 'multiagent', 'system', 'multiagent', 'simulation', 'goal', 'abm', 'search', 'explanatory', 'insight', 'collective', 'behavior', 'agent',

In [49]:
#turning lemmatized words into list of lists
corpus = []

for word in text_tokens:
    corpus.append(word.split())
    
len(corpus)

173825

In [53]:
#creating the Word2Vec model
model = Word2Vec(corpus,  window = 5, min_count = 2, sg = 1)

In [56]:
len(tokens)
for word in tokens[:5]:
    print(word)

nan
agentbased
model
abm
computational


In [65]:
for word in tokens:
    try:
        sims = model.wv.most_similar(word)
    except:
        continue
    for similar in sims:
            item = {'key' : word , 'similar' : similar[0]}
            #outputcoll.insert_one(item)
